In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
ymdb_output_final = '../../../Data/database/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'
target_smiles_path = '../../../Results/not_lipid/target_smiles_not_lipid_complex.pickle'
target_model_smiles_path = '../../../Results/not_lipid/yeast_met_not_lipid.pickle'
yeast8_metabolites_path = '../../../Data/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'
uptake_met_path = '../../../Data/database/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 5
num_process = 60
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid/top{num}_{cut_off}_add_no_ec_re/'
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_add_no_ec_re/'

###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/not_lipid/retrosys_smiles_calculate_similarity_{cut_off}_add_no_ec_re.json'
retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
# rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
# YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_uptake_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
# YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
# rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
# rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'

In [3]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(cut_off_retrosynthesis_path):
    os.makedirs(cut_off_retrosynthesis_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [4]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')


total_smiles 3680
total_smiles 3679
total_smiles 3679
total_smiles 2222


29161it [00:02, 9816.75it/s] 


11134


29161it [00:03, 7318.49it/s] 

16151


In [5]:
# retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
# retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
# retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

In [6]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)

# choose top num similar metabolites
retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

100%|██████████| 11134/11134 [15:52<00:00, 11.69it/s]


### get rxndb

In [7]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)

100%|██████████| 29161/29161 [01:52<00:00, 260.19it/s]


In [8]:
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 60/60 [00:13<00:00,  4.41it/s]


In [9]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)

100%|██████████| 29161/29161 [09:46<00:00, 49.75it/s]  


### retrosynthesis result screen

In [10]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 29161/29161 [00:06<00:00, 4475.65it/s]


rxndb: 11061301
rxndb_drop: 123661


In [11]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(123661, 9)


In [12]:
with open(rxndb_all_path,'r') as f:
    rxndb = json.load(f)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(123661, 9)


In [13]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)

100%|██████████| 100/100 [00:00<00:00, 154.03it/s]


number of metabolites in RXNDB: 5114


In [14]:

#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)

In [15]:
#calculate the similarity of the metabolites

smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)

100%|██████████| 5114/5114 [04:53<00:00, 17.44it/s]


In [16]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

100%|██████████| 100/100 [00:04<00:00, 21.71it/s]


Iteration 1 - Current success count: 4367
Iteration 1 - Current success_rxndbid count: 93576


100%|██████████| 100/100 [00:05<00:00, 19.25it/s]


Iteration 2 - Current success count: 4628
Iteration 2 - Current success_rxndbid count: 98588


100%|██████████| 100/100 [00:05<00:00, 19.00it/s]


...

Iteration 5 - Current success count: 4683
Iteration 5 - Current success_rxndbid count: 100995


100%|██████████| 100/100 [00:05<00:00, 18.54it/s]


Iteration 6 - Current success count: 4689
Iteration 6 - Current success_rxndbid count: 101106


100%|██████████| 100/100 [00:05<00:00, 19.02it/s]


Iteration 7 - Current success count: 4689
Iteration 7 - Current success_rxndbid count: 101106
final success 4689
final success_rxndbid 101106


In [17]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)

success number: 221
fail number: 354


In [18]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)

(101106, 10)
(101106, 10)
(101106, 5)
(101106, 5)
